In [1]:
import numpy as np
#from matplotlib import pyplot as plt
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
#import cv2
import os
#from sklearn import utils
import json
from keras.optimizers import Adam
import tensorflow as tf
from keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Cargamos los tensores y otros datos necesarios para el modelo
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/X_train.npy')
#X_test = np.load('/content/drive/MyDrive/Colab Notebooks/X_test.npy')
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/y_train.npy')
#y_test = np.load('/content/drive/MyDrive/Colab Notebooks/y_test.npy')
target_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]
num_targets = len(target_names)

In [3]:
# SEXTO MODELO:
# Transfer learning, como en el modelo4, pero con otra red preentrenada de Keras:
# InceptionResNetV2

# InceptionResNetV2 es similar a InceptionV3, que es el modelo con mejor rendimiento de esta práctica.
# Pero tiene mejores resultados de accuracy (según https://keras.io/api/applications/) y usa más parámetros.

# Se usará fine tuning.

base_model = InceptionResNetV2(weights='imagenet',
                   include_top=False, # Desactivamos la última capa densa de salida
                   input_shape=(150, 150, 3))

for layer in base_model.layers:
    print("Nombre layer {}".format(layer.name))
    print("\tEs Trainable ? {}".format(layer.trainable))

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Nombre layer input_layer
	Es Trainable ? True
Nombre layer conv2d
	Es Trainable ? True
Nombre layer batch_normalization
	Es Trainable ? True
Nombre layer activation
	Es Trainable ? True
Nombre layer conv2d_1
	Es Trainable ? True
Nombre layer batch_normalization_1
	Es Trainable ? True
Nombre layer activation_1
	Es Trainable ? True
Nombre layer conv2d_2
	Es Trainable ? True
Nombre layer batch_normalization_2
	Es Trainable ? True
Nombre layer activation_2
	Es Trainable ? True
Nombre layer max_pooling2d
	Es Trainable ? True
Nombre layer conv2d_3
	Es Trainable ? True
Nombre layer batch_normalization_3
	Es Trainable ? True
Nombre layer activation_3
	Es Trainable ? True
Nombre layer conv2d_4
	Es Trainable ? True
Nombre layer batch_normalization_4
	Es Trainable ? True
Nombre layer activation_4
	Es Trainable ? True
Nombre layer max_pooling2d_1
	Es Trainable ? True
Nombre layer conv2d_8
	Es Trainable ? True
Nombre layer batch_normalization_8
	

In [4]:
# Inicialmente, congelamos todas las capas
base_model.trainable = False

# Entrenaremos solo las últimas capas
limite_train = len(base_model.layers) - 30
for layer in base_model.layers[:limite_train]:
    layer.trainable = False
for layer in base_model.layers[limite_train:]: # Entrenamos las últimas 30 capas
    layer.trainable = True


In [9]:
model6 = Sequential([ #Añadimos las capas densas
    base_model,
    Flatten(),
    Dense(256, activation='relu'), # Capa densa 1
    Dropout(0.5), # Añadimos un dropout
    Dense(num_targets, activation='softmax') # Capa de salida final
])

In [10]:
model6.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
%%time

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

hist_model6 = model6.fit(X_train, y_train,
                  batch_size = 32,
                  epochs = 15,
                  validation_split = 0.20,
                  callbacks  = [early_stopping])

Epoch 1/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 499s 4s/step - accuracy: 0.8543 - loss: 0.5284 - val_accuracy: 0.8647 - val_loss: 2.0683
Epoch 2/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 487s 4s/step - accuracy: 0.9350 - loss: 0.4530 - val_accuracy: 0.8658 - val_loss: 1.0538
Epoch 3/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 504s 4s/step - accuracy: 0.9570 - loss: 0.2342 - val_accuracy: 0.8974 - val_loss: 0.6881
Epoch 4/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 501s 4s/step - accuracy: 0.9663 - loss: 0.1391 - val_accuracy: 0.8726 - val_loss: 0.7369
Epoch 5/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 502s 4s/step - accuracy: 0.9632 - loss: 0.1305 - val_accuracy: 0.8760 - val_loss: 0.7774
Epoch 6/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 455s 4s/step - accuracy: 0.9753 - loss: 0.0921 - val_accuracy: 0.8963 - val_loss: 0.6602
CPU times: user 1h 15min 18s, sys: 1min 30s, total: 1h 16min 49s
Wall time: 49min 15s


In [12]:
model6.save('/content/drive/MyDrive/Colab Notebooks/model6.h5')
with open('/content/drive/MyDrive/Colab Notebooks/hist_model6.json', 'w') as f:
    json.dump(hist_model6.history, f)